In [3]:
import sys
import os

import pandas as pd
import matplotlib.pyplot as plt

import textwrap as tw
import datetime
from datetime import timedelta

# preamble for jupyter notebook and django
import django
import platform

if platform.node() == "srv-mcc-apsis":
    sys.path.append("/home/galm/software/django/tmv/BasicBrowser/")
    #sys.path.append('/home/leey/tmv/BasicBrowser/')
else:
    # local paths
    sys.path.append('/home/leey/Documents/Data/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import twitter.models as tm
from tmv_app.models import *

#import parliament.models as pm
#from parliament.tasks import do_search, run_tm
import cities.models as cmodels

from django.contrib.auth.models import User
from django.core import serializers
from django.db.models import Sum, Count, Q
from django.db.models.fields import DateField
from django.db.models.functions import Cast

In [2]:
import matplotlib as mpl

mpl.rcParams['font.size'] = 6

dsize = 7.0866142

In [3]:
# Coal tweets total
kt = tm.Status.objects.filter(text__icontains="Kohle")
kt.count()

849359

In [4]:
users = tm.User.objects
users.count()

5325944

In [5]:
userst = tm.User.objects.filter(monitoring=True)
userst.count()

690

In [6]:
# Coal tweets by parliamentarians
kp = tm.Status.objects.filter(author__monitoring=True,text__icontains="Kohle")
kp.count()

10149

In [ ]:
# Average length of tweets

In [9]:
s = tm.Status.objects.filter(
    text__icontains="kohle",
    author__monitoring=True,
    #author__person__isnull=True
).exclude(
    text__iregex="Kohler\W"
)
ats = s.values('author__name').annotate(
    n = Count('pk')
).order_by('-n')

kommission = set(s.distinct('author__id').values_list('author__id',flat=True))
kommission = tm.User.objects.filter(id__in=kommission)

kommission.count()

354

In [14]:
pd.DataFrame(ats)

,author__name,n
0,Initiative Buirer für Buir,2251
1,Hubert Weiger,580
2,BUND,469
3,Martin Kaiser,428
4,"Dr. Julia Verlinden, MdB",425
5,Oliver Krischer,402
6,Lorenz Gösta Beutin,292
7,Annalena Baerbock,278
8,IG BCE,217
9,Jürgen Trittin,147


---

In [5]:
def tweet_time_graph(s, ax, years = None):
    if s.status_set.exists():
        statuses = s.status_set.all()
        if years:
            start = django.utils.timezone.make_aware(datetime.datetime.now()) - datetime.timedelta(days=years*365)
            statuses = statuses.filter(created_at__gt=start)
        
        byday = statuses.annotate(
            date_only=Cast('created_at', DateField())
        ).values('date_only').annotate(
            n = Count('pk')
        ).order_by('date_only')

        df = pd.DataFrame.from_dict(list(byday))

        df.head()

        df['date_only'] = pd.to_datetime(df['date_only'])

        df.plot('date_only','n', style=".", ax=ax, ms=3)
        ax.set_title(s.string)
        return df

In [1]:
terms =  [
    '#Kohlekommission',
    '#Kohleausstieg',
    #'#Klimakrise',
    #'#Klimawandel',
    #'#Klimaschutz',
    '#Hambibleibt',
]

In [4]:
# number of tweets containing terms
# how to not double count?
kht = tm.Status.objects.filter(Q(text__icontains="Kohlekommission")|Q(text__icontains="Kohleausstieg")|Q(text__icontains="Hambibleibt"))
kht.count()

228349

In [ ]:
# number of users 

In [ ]:
fig = plt.figure(dpi=200,figsize=(dsize,dsize))

i=0
fax = None
for t in terms:
    s, created = tm.TwitterSearch.objects.get_or_create(string=t)
    print(f"\n{s.string}: {s.status_set.count()}")
    if s.status_set.count() > 1:
        ms = s.status_set.order_by('-retweets_count').first()
        print(f"most retweeted: {ms.retweets_count}")
        print(f"{ms.author} - {ms.created_at}")
        print(ms.text)
        
        i+=1
        
        if fax:
            ax = fig.add_subplot(2,3,i,sharex=ax)
        else:
            ax = fig.add_subplot(2,3,i)
            fax = ax
        tweet_time_graph(s, ax)


plt.minorticks_off()
fig.tight_layout()